# Лабораторная работа 1
## Системы линейных алгебраических уравнений

### 1. Реализовать алгоритм LU - разложения матриц (с выбором главного элемента) в виде программы. Используя разработанное программное обеспечение, решить систему линейных алгебраических уравнений (СЛАУ). Для матрицы СЛАУ вычислить определитель и обратную матрицу.

In [4]:
import numpy as np
from math import log10, atan, pi, sin, cos, sqrt

In [5]:
def get_script(s, script='sup'):
    if script == 'sup':
        return s.translate(s.maketrans('0123456789', '⁰¹²³⁴⁵⁶⁷⁸⁹'))
    if script == 'sub':
        return s.translate(s.maketrans('0123456789', '₀₁₂₃₄₅₆₇₈₉'))
    return s

In [6]:
def polynom2str(a, x = 'x', script='sup'):
    s = ''
    frm = 0
    while a[frm]==0: frm += 1
    first_done = False
    for i in range(frm, len(a)):
        if a[i] == 0: continue
        deg = len(a) - i - 1
        if first_done:
            s += ('+ ' if a[i] > 0 else '- ')
        elif a[i] < 0:
            s += '-'
        first_done = True
        if abs(a[i]) != 1 or deg == 0:
            s += '%g'%abs(a[i])
        if script == 'sup':
            if deg > 0:
                s += x
                if deg != 1:
                    s += get_script(str(deg), script)
                s += ' '
        else:
            s += x + get_script(str(i+1), script) + ' '
    return s if s!='' else '0'

In [7]:
def print_system(a, b):
    for i in range(len(a)):
        print('{', polynom2str(a[i], script='sub'), '=', b[i])

In [8]:
def print_x_solution(x, sym = 'x'):
    for i in range(1, len(x)+1):
        print(f'{sym}{get_script(str(i), "sub")} = %g;' % round(x[i-1], 3), end='  ')
    print()

In [9]:
def print_vec_solution(x, sym = 'x'):
    for i in range(1, len(x)+1):
        print(f'{sym}{get_script(str(i), "sub")} = {x[i-1]};')
    print()

In [10]:
def print_comp_solution(x, sym = 'x'):
    for i in range(1, len(x)+1):
        s = str(round(x[i-1][0], 4))
        if x[i-1][1] != 0:
            if x[i-1][1] < 0: s += ' - ' 
            else: s += ' + '
            s += str(round(abs(x[i-1][1]), 4)) + ' i'
        print(f'{sym}{get_script(str(i), "sub")} = {s};')
    print()

In [11]:
class LU:
    EPS = 1e-6

    def __init__(self, U):
        L = np.eye(len(U), dtype=float)
        isDetNeg = False
        permut = np.array(range(len(U)))
        for i in range(len(U)):
            max_idx = i
            for j in range(i + 1, len(U)):
                if abs(U[max_idx][i]) < abs(U[j][i]):
                    max_idx = j
            if max_idx != i:
                U[[i, max_idx]] = U[[max_idx, i]]
                L[[i, max_idx]] = L[[max_idx, i]]
                L[:, [i, max_idx]] = L[:, [max_idx, i]]
                isDetNeg = not isDetNeg
                permut[[i, max_idx]] = permut[[max_idx, i]]
            if abs(U[i][i]) < self.EPS: continue
            for j in range(i + 1, len(U)):
                mu = U[j][i] / U[i][i]
                L[j][i] = mu
                for k in range(len(U)):
                    U[j][k] -= mu * U[i][k]
        det = U.diagonal().prod()
        if isDetNeg: det = -det
        self._permut = permut
        self.L = L
        self.U = U
        self.det = det

    def solve(self, b):
        b = np.array([ b[pi] for pi in self._permut ], dtype=float)
        z = np.array([0] * len(b), dtype=float)
        for i in range(len(b)):
            z[i] = b[i]
            for j in range(i):
                z[i] -= self.L[i, j] * z[j]
        x = np.array([0] * len(b), dtype=float)
        for i in range(len(b)-1, -1, -1):
            if abs(self.U[i, i]) < self.EPS: continue
            x[i] = z[i]
            for j in range(len(b)-1, i, -1):
                x[i] -= x[j] * self.U[i, j]
            x[i] /= self.U[i, i]
        return x
    
    def inverse(self):
        n = len(self.L)
        ret = np.matrix([[0] * n] * n, dtype=float)
        for i in range(n):
            b = np.array([0] * n, dtype=float)
            b[i] = 1
            ret[:, i] = np.matrix(self.solve(b)).T
        return ret

In [12]:
A = np.array([[-1, -7, -3, -2],
              [-8, 1, -9, 0],
              [ 8,  2, -5, -3],
              [-5, 3,  5, -9]], dtype=float)
b = np.array([-12, -60, -91, -43], dtype=float)
print_system(A, b)

{ -x₁ - 7x₂ - 3x₃ - 2x₄  = -12.0
{ -8x₁ + x₂ - 9x₃  = -60.0
{ 8x₁ + 2x₂ - 5x₃ - 3x₄  = -91.0
{ -5x₁ + 3x₂ + 5x₃ - 9x₄  = -43.0


In [13]:
lu = LU(A)
print('Решение системы: ', end='')
print_x_solution(lu.solve(b))
print('Определитель матрицы:', round(lu.det, 3))
print('Обратная матрица:\n', lu.inverse())

Решение системы: x₁ = -2;  x₂ = -4;  x₃ = 8;  x₄ = 9;  
Определитель матрицы: -10339.0
Обратная матрица:
 [[ 0.00203114 -0.04807041  0.06364252 -0.02166554]
 [-0.12215882  0.03394912  0.02949995  0.01731309]
 [-0.01537866 -0.06460973 -0.05329336  0.02118193]
 [-0.05039172  0.00212787 -0.05513106 -0.08153593]]


In [14]:
l = np.matrix(lu.L)
u = np.matrix(lu.U)
l * u

matrix([[-8.,  1., -9.,  0.],
        [-1., -7., -3., -2.],
        [ 8.,  2., -5., -3.],
        [-5.,  3.,  5., -9.]])

### 2. Реализовать метод прогонки в виде программы, задавая в качестве входных данных ненулевые элементы матрицы системы и вектор правых частей. Используя разработанное программное обеспечение, решить СЛАУ с трехдиагональной матрицей.

In [15]:
import math

print("Введите элементы нижней, главной, верхней диагоналей и вектор решений:")
a = list(map(int, input().split()))
b = list(map(int, input().split()))
c = list(map(int, input().split()))
d = list(map(int, input().split()))

p = [-c[0] / b[0]]
q = [d[0] / b[0]]
n = len(d)

x = [0]*n


for i in range(1, n):
    p.append(-c[i]/(b[i] + a[i]*p[i-1]))
    q.append((d[i] - a[i]*q[i-1])/(b[i] + a[i]*p[i-1]))

x[n-1] = q[n-1]

for i in range(n-1, -1, -1):
    x[i-1] = p[i-1]*x[i] + q[i-1]

print("\nОтвет:")
print (x)

Введите элементы нижней, главной, верхней диагоналей и вектор решений:
0 -5 -5 -9 1
8 22 -11 -15 7
4 8 1 1 0
48 125 -43 18 -23

Ответ:
[3.0, 6.0, 0.9999999999999998, -2.0, -3.0]


### 3. Реализовать метод простых итераций и метод Зейделя в виде программ, задавая в качестве входных данных матрицу системы, вектор правых частей и точность вычислений. Используя разработанное программное обеспечение, решить СЛАУ. Проанализировать количество итераций, необходимое для достижения заданной точности.

In [16]:
def L2_norm(X):
    """
    Count ||X||_2
    """
    n = X.shape[0]
    l2_norm = 0
    for i in range(n):
        l2_norm += X[i] * X[i]
    return math.sqrt(l2_norm)


def solve_iterative(A, b, eps):
    """
    Uses iterative method to solve Ax=b
    Returns x and number of iterations
    """
    n = A.shape[0]

    # Step 1. Ax=b -> x = alpha * x + beta
    alpha = np.zeros_like(A, dtype='float')
    beta = np.zeros_like(b, dtype='float')
    for i in range(n):
        for j in range(n):
            if i == j:
                alpha[i][j] = 0
            else:
                alpha[i][j] = -A[i][j] / A[i][i]

        beta[i] = b[i] / A[i][i]

    # Step 2. Iterating
    iterations = 0
    cur_x = np.copy(beta)
    converge = False
    while not converge:
        prev_x = np.copy(cur_x)
        cur_x = alpha @ prev_x + beta
        iterations += 1
        converge = L2_norm(prev_x - cur_x) <= eps
    return cur_x, iterations


def seidel_multiplication(alpha, x, beta):
    """
    Count alhpa * x + beta for seidel method
    """
    res = np.copy(x)
    for i in range(alpha.shape[0]):
        res[i] = beta[i]
        for j in range(alpha.shape[1]):
            res[i] += alpha[i][j] * res[j]
    return res


def solve_seidel(A, b, eps):
    """
    Uses Seidel method to solve Ax=b
    Returns x and number of iterations
    """
    n = A.shape[0]

    # Step 1. Ax=b -> x = alpha * x + beta
    alpha = np.zeros_like(A, dtype='float')
    beta = np.zeros_like(b, dtype='float')
    for i in range(n):
        for j in range(n):
            if i == j:
                alpha[i][j] = 0
            else:
                alpha[i][j] = -A[i][j] / A[i][i]

        beta[i] = b[i] / A[i][i]

    # Step 2. Iterating
    iterations = 0
    cur_x = np.copy(beta)
    converge = False
    while not converge:
        prev_x = np.copy(cur_x)
        cur_x = seidel_multiplication(alpha, prev_x, beta)
        iterations += 1
        converge = L2_norm(prev_x - cur_x) <= eps
    return cur_x, iterations


if __name__ == '__main__':
    n = int(input('Enter the size of matrix: '))

    print('Enter matrix A')
    A = [[] for _ in range(n)]
    for i in range(n):
        row = list(map(int, input().split()))
        A[i] = row
    A = np.array(A, dtype='float')
    print('Enter vector b')
    b = np.array(list(map(int, input().split())), dtype='float')
    eps = float(input('Enter epsilon: '))

    print('Iteration method')
    x_iter, i_iter = solve_iterative(A, b, eps)
    print(x_iter)
    print('Iterations:', i_iter)
    print()

    print('Seidel method')
    x_seidel, i_seidel = solve_seidel(A, b, eps)
    print(x_seidel)
    print('Iterations:', i_seidel)

Enter the size of matrix: 4
Enter matrix A
26 -9 -8 8
9 -21 -2 8
-3 2 -18 8
1 -6 -1 11
Enter vector b
20 -164 140 -81
Enter epsilon: 0.001
Iteration method
[ 1.999822    8.00019653 -8.99996642 -4.0001716 ]
Iterations: 30

Seidel method
[ 1.99970731  7.99978167 -9.00013572 -4.00010482]
Iterations: 8


### 1.4. Реализовать метод вращений в виде программы, задавая в качестве входных данных матрицу и точность вычислений. Используя разработанное программное обеспечение, найти собственные значения и собственные векторы симметрических матриц. Проанализировать зависимость погрешности вычислений от числа итераций. 

In [17]:
def find_max_upper_element(X):
    """
    Find coords of max element by absolute value above the main diagonal
    Returns i, j of max element
    """
    n = X.shape[0]
    i_max, j_max = 0, 1
    max_elem = abs(X[0][1])
    for i in range(n):
        for j in range(i + 1, n):
            if abs(X[i][j]) > max_elem:
                max_elem = abs(X[i][j])
                i_max = i
                j_max = j
    return i_max, j_max

In [18]:
def matrix_norm(X):
    """
    Calculates L2 norm for elements above the main diagonal
    """
    norm = 0
    for i in range(n):
        for j in range(i + 1, n):
            norm += X[i][j] * X[i][j]
    return np.sqrt(norm)

In [19]:
def rotation_method(A, eps):
    """
    Find eigen values and eigen vectors using rotation method
    Returns eigen values, eigen vectors, number of iterations
    """
    n = A.shape[0]
    A_i = np.copy(A)
    eigen_vectors = np.eye(n)
    iterations = 0

    while matrix_norm(A_i) > eps:
        i_max, j_max = find_max_upper_element(A_i)
        if A_i[i_max][i_max] - A_i[j_max][j_max] == 0:
            phi = np.pi / 4
        else:
            phi = 0.5 * np.arctan(2 * A_i[i_max][j_max] / (A_i[i_max][i_max] - A_i[j_max][j_max]))

        # create rotation matrix
        U = np.eye(n)
        U[i_max][j_max] = -np.sin(phi)
        U[j_max][i_max] = np.sin(phi)
        U[i_max][i_max] = np.cos(phi)
        U[j_max][j_max] = np.cos(phi)

        A_i = U.T @ A_i @ U
        eigen_vectors = eigen_vectors @ U
        iterations += 1

    eigen_values = np.array([A_i[i][i] for i in range(n)])
    return eigen_values, eigen_vectors, iterations


In [20]:
if __name__ == '__main__':
    n = int(input('Enter the size of matrix: '))

    print('Enter matrix A')
    A = [[] for _ in range(n)]
    for i in range(n):
        row = list(map(int, input().split()))
        A[i] = row
    A = np.array(A, dtype='float')
    eps = float(input('Enter epsilon: '))

    eig_values, eig_vectors, iters = rotation_method(A, eps)
    print('Eigen values:', eig_values, "\n")
    print('Eigen vectors')
    print(eig_vectors, "\n")
    print('Iterations:', iters, "\n")

Enter the size of matrix: 3
Enter matrix A
8 2 -1
2 -5 -8
-1 -8 -5
Enter epsilon: 0.001
Eigen values: [  8.79894069 -13.02410419   2.2251635 ] 

Eigen vectors
[[ 0.93869992 -0.03407344  0.34304732]
 [ 0.26434633  0.70987678 -0.65283687]
 [-0.22127693  0.70350122  0.67536846]] 

Iterations: 5 



In [21]:
    for i in range(n):
        #print(eig_vectors.T)
        h = eig_vectors.T
        #print(eig_values[i])
        print(i, 'Testing1:', A.T.dot(h[i]))
        print(i, 'Testing2:', eig_values[i] * h[i], "\n")
        i = i + 1

0 Testing1: [ 8.25956894  2.32588361 -1.94708593]
0 Testing2: [ 8.2595649   2.32596771 -1.94700259] 

1 Testing1: [ 0.44366483 -9.2455405  -9.16244686]
1 Testing2: [ 0.443776   -9.24550909 -9.16247317] 

2 Testing1: [ 0.76333638 -1.45266868  1.50280534]
2 Testing2: [ 0.76333638 -1.45266878  1.50280525] 



### 1.5. Реализовать алгоритм QR – разложения матриц в виде программы. На его основе разработать программу, реализующую QR – алгоритм решения полной проблемы собственных значений произвольных матриц, задавая в качестве входных данных матрицу и точность вычислений. С использованием разработанного программного обеспечения найти собственные значения матрицы.

In [22]:
def sign(x):
    return -1 if x < 0 else 1 if x > 0 else 0

def L2_norm(vec):
    """
    Counts L2 norm of a vector
    """
    ans = 0
    for num in vec:
        ans += num * num
    return np.sqrt(ans)

In [23]:
def get_householder_matrix(A, col_num):
    """
    Get householder matrix for iteration of QR decomposition
    Returns householder matrix H
    """
    n = A.shape[0]
    v = np.zeros(n)
    a = A[:, col_num]
    v[col_num] = a[col_num] + sign(a[col_num]) * L2_norm(a[col_num:])
    for i in range(col_num + 1, n):
        v[i] = a[i]
    v = v[:, np.newaxis]
    H = np.eye(n) - (2 / (v.T @ v)) * (v @ v.T)
    return H

In [24]:
def QR_decomposition(A):
    """
    Make QR decomposition: A = QR
    Returns Q, R
    """
    n = A.shape[0]
    Q = np.eye(n)
    A_i = np.copy(A)

    for i in range(n - 1):
        H = get_householder_matrix(A_i, i)
        Q = Q @ H
        A_i = H @ A_i
    return Q, A_i

In [25]:
def get_roots(A, i):
    """
    Get roots of system of two equations (i and i+1) of matrix A
    """
    n = A.shape[0]
    a11 = A[i][i]
    a12 = A[i][i + 1] if i + 1 < n else 0
    a21 = A[i + 1][i] if i + 1 < n else 0
    a22 = A[i + 1][i + 1] if i + 1 < n else 0
    return np.roots((1, -a11 - a22, a11 * a22 - a12 * a21))

In [26]:
def is_complex(A, i, eps):
    """
    Check if i and (i+1)-th eigen values are complex
    """
    Q, R = QR_decomposition(A)
    A_next = np.dot(R, Q)
    lambda1 = get_roots(A, i)
    lambda2 = get_roots(A_next, i)
    return abs(lambda1[0] - lambda2[0]) <= eps and abs(lambda1[1] - lambda2[1]) <= eps

In [27]:
def get_eigen_value(A, i, eps):
    """
    Get i-th (and (i+1)-th if complex) eigen value of matrix A.
    Returns eigen value(s) and matrix A_i for the next iterations
    """
    A_i = np.copy(A)
    while True:
        Q, R = QR_decomposition(A_i)
        A_i = R @ Q
        if L2_norm(A_i[i + 1:, i]) <= eps:
            return A_i[i][i], A_i
        elif L2_norm(A_i[i + 2:, i]) <= eps and is_complex(A_i, i, eps):
            return get_roots(A_i, i), A_i

In [28]:
def get_eigen_values_QR(A, eps):
    """
    Count all eigen values of A using QR decomposition
    """
    n = A.shape[0]
    A_i = np.copy(A)
    eigen_values = []

    i = 0
    while i < n:
        cur_eigen_values, A_i_plus_1 = get_eigen_value(A_i, i, eps)
        if isinstance(cur_eigen_values, np.ndarray):
            # complex
            eigen_values.extend(cur_eigen_values)
            i += 2
        else:
            # real
            eigen_values.append(cur_eigen_values)
            i += 1
        A_i = A_i_plus_1
    return eigen_values

In [30]:
if __name__ == '__main__':
    n = int(input('Enter the size of matrix: '))

    print('Enter matrix A')
    A = [[] for _ in range(n)]
    for i in range(n):
        row = list(map(int, input().split()))
        A[i] = row
    A = np.array(A, dtype='float')
    eps = float(input('Enter epsilon: '))

    eig_values = get_eigen_values_QR(A, eps)
    print('Eigen values:', eig_values)

Enter the size of matrix: 3
Enter matrix A
-4 -6 -3
-1 5 -5
6 2 5
Enter epsilon: 0.001
Eigen values: [7.111398329519387, (-0.5557251391279904+3.8205781617612535j), (-0.5557251391279904-3.8205781617612535j)]
